In [ ]:
!jupyter-nbconvert --to python --template python_clean codesign.ipynb

In [1]:
import configparser
import argparse
from distutils import util
import subprocess
import shlex
import tempfile
from pathlib import Path
from time import sleep

In [2]:
def runcommand (cmd):
    '''run an external command and return exit code, stdout and stderr
    
    https://gist.github.com/jotaelesalinas/f809d702e4d3e24b19b77b83c9bf5d9e'''
    proc = subprocess.Popen(cmd,
                            stdout=subprocess.PIPE,
                            stderr=subprocess.PIPE,
                            shell=True,
                            universal_newlines=True)
    std_out, std_err = proc.communicate()
    return proc.returncode, std_out, std_err

In [3]:
def get_config(file=None, default_config=None, filename='codesign.ini'):
    config = configparser.ConfigParser()
    
    if file:
        print (f'using configuration file: {file}')
        config.read(file)
    elif default_config:
        config.read_dict(default_config)
        print(f'writing default config file: {filename}')
        try:
            with open(filename, 'w') as blank_config:
                config.write(blank_config)
        except OSError as e:
            print(f'could not create {filename} due to error: {e}')
        return {}
    else:
        return {}

    return {s:dict(config.items(s)) for s in config.sections()}

In [4]:
def get_args():
    parser = argparse.ArgumentParser(description='Commandline Parser')
    
    parser.add_argument('-n', '--new', dest='new_config', 
                        action='store_true', default=False,
                       help='create a new sample configuration with name "codesign.ini" in current directory')
    
    parser.add_argument('config', nargs='?', type=str, default=None,
                       help='configuration file to use when codesigning')
    
    parser.add_argument('-s', '--sign', dest='sign_only',
                       action='store_true', default=None,
                       help='sign the executables, but take no further action (can be combined with -p, -o, -t)')
    
    
#     known_args, unknown_args = parser.parse_known_args()
    args = parser.parse_args()
#     return(known_args, unknown_args)
    return args

In [5]:
def validate_config(config, expected_keys):
    missing = {}
    for section, keys in expected_keys.items():
        if not section in config.keys():
            missing[section] = expected_keys[section]
            continue
        for key in keys:
            if not key in config[section].keys():
                if not section in missing:
                    missing[section] = {}
                missing[section][key] = keys[key]
                
    if missing:
        print(f'Config file "{args.config}" is missing values:')
        for section, values in missing_values.items():
            print(f'[{section}]')
            for k, v in values.items():
                print(f'\t{k}: {v}')
                    
    return missing

In [6]:
import sys

In [7]:
sys.argv

['/Users/aaronciuffo/.local/share/virtualenvs/pyPDF_split-aG0Oa909/lib/python3.8/site-packages/ipykernel_launcher.py',
 '-f',
 '/Users/aaronciuffo/Library/Jupyter/runtime/kernel-ca1c93ba-1b8e-44ea-b9c7-6634ff6aaeec.json']

In [ ]:
default_args = sys.argv

In [11]:
sys.argv = []

In [13]:
sys.argv.append('project.ini')

In [14]:
def run_command(command_list):
    cmd = subprocess.Popen(shlex.split(' '.join(command_list)), 
                    stdout=subprocess.PIPE,
                    stderr=subprocess.PIPE)
    stderr, stdout = cmd.communicate()
    return cmd.returncode, stderr, stdout

In [15]:
def sign(config):
    
    try:
        entitlements = util.strtobool(config['package_details']['entitlements'])
    except (AttributeError, ValueError):
        entitlements = config['package_details']['entitlements']
    
    if (not entitlements) or (entitlements == 'None'):
        entitlements = None
        
    config['package_details']['entitlements'] = entitlements

    args = {
        'command': 'codesign',
        'args': '--deep --force --timestamp --options=runtime',
        'entitlements': f'--entitlements {config["package_details"]["entitlements"]}' if config["package_details"]["entitlements"] else None,
        'signature': f'--sign {config["identification"]["application_id"]}',
        'files': ' '.join(config['package_details']['file_list'])
    }
    
    final_list = [i if i is not None else '' for k, i in args.items()]
    print(f'signing files: {args["files"]}')
    
    return_code, stdout, stderr = run_command(final_list)
    for each in (stderr, stdout):
        print(str(each, 'utf-8'))
    return return_code, stdout, stderr
    

In [16]:
def package(config):
    pkg_temp = tempfile.TemporaryDirectory()
    pkg_temp_path = Path(pkg_temp.name)
    
    install_path = config['package_details']['installation_path']
    for file in config['package_details']['file_list']:
        file_name = Path(file).name
        return_code, stderr, stdout = run_command(shlex.split(f'ditto {file} {pkg_temp_path/file_name}'))
        if return_code > 0:
            pkg_temp.cleanup()
            return return_code, stderr, stdout
    
    args = {
        'command': 'productbuild',
        'identifier': f'--identifier {config["package_details"]["bundle_id"]}.pkg',
        'signature': f'--sign {config["identification"]["installer_id"]}',
        'args': '--timestamp',
        'root': f'--root {pkg_temp_path} / ./{config["package_details"]["package_name"]}.pkg'
        
    }
    
    print(f'packaging {config["package_details"]["package_name"]}.pkg')
    final_list = [i if i is not None else '' for k, i in args.items()]
    return_code, stderr, stdout = run_command(final_list)
    
    for each in (stderr, stdout):
        if len(each) > 0:
            print(str(each, 'utf-8'))
        
    pkg_temp.cleanup()    
    return return_code, stdout, stderr

In [25]:
def notarize(config):
    notarize_args = {
        'command': 'xcrun altool',
        'args': '--notarize-app',
        'bundle_id': f'--primary-bundle-id {config["package_details"]["bundle_id"]}',
        'username': f'--username={config["identification"]["apple_id"]}',
        'password': f'--password {config["identification"]["password"]}',
        'file': f'--file ./{config["package_details"]["package_name"]}.pkg'
    }
    
 
    
    final_list = [i for k, i, in notarize_args.items()]
    return_code, stdout, stderr = run_command(final_list)
            
   
    return return_code, stdout, stderr

In [32]:
s = notarize(c)

In [48]:
def check_notarization(stdout, config):
    notarize_max_check = config['main']['notrarize_max_check']
    notarize_check = 0
    success = {}
    notarized = False
    
    
    uuids = []
    for line in str(stdout[1], 'utf-8').splitlines():
        if 'requestuuid' in line.lower():
            my_id = line.split('=')
            uuids.append(my_id[1].strip())    
    

    while not notarized:
        print('checking notarization status')
        notarize_check += 1
        print(f'check: {notarize_check}')
        check_args = {
            'command': 'xcrun altool',
            'info': f'--notarization-info {uuids[0]}',
            'username': f'--username {config["identification"]["apple_id"]}',
            'password': f'--password {config["identification"]["password"]}'
        }
        final_list = [i for k, i in check_args.items()]
        return_code, stdout, stderr = run_command(final_list)

        if stdout:
            lines = str(stdout, 'utf-8').splitlines()

            for l in lines:
                if 'status' in l.lower():
                    vals = l.split(':')
                    success[vals[0].strip()] = vals[1].strip()

            if success:
                notarized=True    
            else:
                if notarize_check >= notarize_max_check-1:
                    break
                sleep_timer = config['main']['notarize_timer']*notarize_check
                print(f'sleeping for {sleep_timer} seconds')
                sleep(sleep_timer)
            
    return success

In [49]:
check_notarization(s, c)

checking notarization status
check: 1


{'Status': 'success', 'Status Code': '0', 'Status Message': 'Package Approved'}

In [22]:
notarize(c)

AttributeError: 'int' object has no attribute 'lower'

In [18]:
# config = c
# uuids = ['320f2c00-82e1-4c33-a599-63ddec405ef3']
# notarize_max_check = config['main']['notrarize_max_check']
# notarize_check = 0
# success = {}
# notarized = False


# while not notarized:
#     print('checking notarization status')
#     notarize_check += 1
#     print(f'check: {notarize_check}')
#     check_args = {
#         'command': 'xcrun altool',
#         'info': f'--notarization-info {uuids[0]}',
#         'username': f'--username {config["identification"]["apple_id"]}',
#         'password': f'--password {config["identification"]["password"]}'
#     }
#     final_list = [i for k, i in check_args.items()]
#     return_code, stdout, stderr = run_command(final_list)

#     if stdout:
#         lines = str(stdout, 'utf-8').splitlines()

#         for l in lines:
#             if 'status' in l.lower():
#                 vals = l.split(':')
#                 success[vals[0].strip()] = vals[1].strip()

#         if success:
#             notarized=True    
#         else:
#             if notarize_check >= notarize_max_check-1:
#                 break
#             sleep_timer = config['main']['notarize_timer']*notarize_check
#             print(f'sleeping for {sleep_timer} seconds')
#             sleep(sleep_timer)
        

In [19]:
def staple(config):
    pass

In [20]:
def main():
    expected_config_keys = {
        'identification': {
            'application_id': 'Unique Substring of Developer ID Application Cert',
            'installer_id': 'Unique Substring of Developer ID Installer Cert',
            'apple_id': 'developer@domain.com',
            'password': '@keychain:App-Specific-Password-Name-In-Keychain',
        },
        'package_details': {
            'package_name': 'nameofpackage',
            'bundle_id': 'com.developer.packagename',
            'file_list': "include_file1, include_file2",
            'installation_path': '/Applications/',
            'entitlements': None,
            'version': '0.0.0'
        }
    }
    
    notarize_timer = 60
    notrarize_max_check = 5
    
    
    args = get_args()

    config = get_config(file=args.config, default_config=expected_config_keys)
    if not config:
        print('no configuration file provided')
        return

    config.update({'main': {
        'notarize_timer': notarize_timer,
        'notrarize_max_check': notrarize_max_check}
                  })
    
    if validate_config(config, expected_config_keys):
        print('exiting')
        return
        
    # split the file list into an actual list
    try:
        file_list = config['package_details']['file_list'].split(',')
        config['package_details']['file_list'] = file_list
    except KeyError:
        pass
    
    return config        
    

In [21]:
c = main()

using configuration file: project.ini


In [ ]:
c